In [1]:
# ==============================================================================
# KOMÓRKA 1: WCZYTANIE DANYCH I SZYBKA NAPRAWA KOLUMN `loc_*`
# ==============================================================================
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import os
import json
import joblib
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

print("TensorFlow version:", tf.__version__)
ARTIFACTS_DIR = '1_Artifacts_Polska'
if not os.path.exists(ARTIFACTS_DIR): os.makedirs(ARTIFACTS_DIR)

# Wczytujemy wyniki ciężkiej pracy - plik .pkl
print("Wczytuję przetworzone dane z pliku 'df_flats_processed.pkl'...")
df_flats = pd.read_pickle('df_flats_processed.pkl')
print(f"Dane wczytane. Liczba ogłoszeń: {len(df_flats)}")

# Wczytujemy też plik z lokalizacjami
LOKALIZACJA_FILE = 'lokalizacja.csv'
LOK_COLUMNS = ['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName']
df_lok = pd.read_csv(LOKALIZACJA_FILE, header=None, names=LOK_COLUMNS, na_values='\\N', low_memory=False)
df_lok['ParentId'] = df_lok['ParentId'].fillna(0).astype('Int64')
lok_dict = df_lok.set_index('Id')['Name'].to_dict()

# =================================================================
# SERCE OPERACJI: SZYBKA NAPRAWA BŁĘDNIE PRZETWORZONYCH KOLUMN
# =================================================================
print("\nNaprawiam kolumny 'loc_*' na podstawie oryginalnej 'locationPath'...")

loc_path_cols = [f'loc_{i}' for i in range(1, 8)]

# Usuwamy stare, błędne kolumny, jeśli istnieją
for col in loc_path_cols:
    if col in df_flats.columns:
        df_flats.drop(col, axis=1, inplace=True)
if 'city_name' in df_flats.columns:
    df_flats.drop('city_name', axis=1, inplace=True)


# Używamy nowej, odpornej metody do stworzenia poprawnych kolumn
path_lists = df_flats['locationPath'].astype(str).str.split(',')
processed_paths = pd.DataFrame(path_lists.tolist(), index=df_flats.index)
if processed_paths.shape[1] < 7:
    for i in range(processed_paths.shape[1], 7):
        processed_paths[i] = None
processed_paths = processed_paths.iloc[:, :7]
processed_paths.columns = loc_path_cols
df_flats[loc_path_cols] = processed_paths.fillna(0).astype(int)

# Tworzymy na nowo 'city_name'
df_flats['city_name'] = df_flats['loc_4'].map(lok_dict)
df_flats.dropna(subset=['city_name'], inplace=True) # Usuwamy ogłoszenia bez przypisanego miasta
print("Kolumny 'loc_*' i 'city_name' zostały naprawione.")


# --- DIAGNOSTYKA PO POPRAWCE ---
print("\n--- Diagnostyka po naprawie ---")
train_data_check = df_flats[(df_flats['loc_5'] != 0) & (df_flats['loc_6'] != 0) & (df_flats['loc_7'] != 0)]
print(f"Liczba wierszy z PEŁNĄ lokalizacją: {len(train_data_check)}")
if len(train_data_check) > 0:
    print("[SUKCES] Dane treningowe zostały poprawnie zidentyfikowane!")
else:
    print("[BŁĄD] Wciąż brak danych treningowych.")

print("\nDane gotowe do treningu modelu.")

TensorFlow version: 2.19.0
Wczytuję przetworzone dane z pliku 'df_flats_processed.pkl'...
Dane wczytane. Liczba ogłoszeń: 1100194

Naprawiam kolumny 'loc_*' na podstawie oryginalnej 'locationPath'...
Kolumny 'loc_*' i 'city_name' zostały naprawione.

--- Diagnostyka po naprawie ---
Liczba wierszy z PEŁNĄ lokalizacją: 153562
[SUKCES] Dane treningowe zostały poprawnie zidentyfikowane!

Dane gotowe do treningu modelu.


In [2]:
# ==============================================================================
# KOMÓRKA 2: TRENING MODELI
# ==============================================================================
print("\n--- Krok 3: Trening Modeli ---")

# Definicje cech, upewnijmy się, że wszystkie kolumny istnieją
numerical_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear']
categorical_features = ['BuldingType']
text_based_features = ['text_district_id', 'text_subdistrict_id', 'text_street_id']
similarity_based_features = ['similar_district_id', 'similar_subdistrict_id', 'similar_street_id']

# Pętla po miastach i trening
for city_name in df_flats['city_name'].unique():
    if pd.isna(city_name): continue
    
    print(f"\n--- Trenowanie modelu dla miasta: {city_name} ---")
    
    df_city = df_flats[df_flats['city_name'] == city_name].copy()
    # Dane treningowe to tylko te z PEŁNĄ lokalizacją
    train_data = df_city[(df_city['loc_5'] != 0) & (df_city['loc_6'] != 0) & (df_city['loc_7'] != 0)].copy()
    
    if len(train_data) < 50:
        print(f"Pominięto {city_name} z powodu zbyt małej ilości danych treningowych ({len(train_data)} ogłoszeń).")
        continue

    # Przygotowanie cech wejściowych (X)
    X_num = train_data[numerical_features].values
    X_cat_dummies = pd.get_dummies(train_data[categorical_features], prefix='type').values
    X_text = train_data[text_based_features].values
    X_sim = train_data[similarity_based_features].values
    
    scaler = MinMaxScaler()
    X_num_scaled = scaler.fit_transform(X_num)
    
    X = np.hstack([X_num_scaled, X_cat_dummies, X_text, X_sim])
    
    # Przygotowanie etykiet (y)
    encoders = {}
    targets = {}
    target_cols = {'district': 'loc_5', 'subdistrict': 'loc_6', 'street': 'loc_7'}

    for name, col in target_cols.items():
        le = LabelEncoder()
        targets[name] = le.fit_transform(train_data[col])
        encoders[name] = {'classes': le.classes_.tolist(), 'mapping': {i: int(cls) for i, cls in enumerate(le.classes_)}}

    # Zapisanie artefaktów
    joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, f'{city_name}_scaler.gz'))
    ohe_columns = pd.get_dummies(train_data[categorical_features], prefix='type').columns.tolist()
    encoders['ohe_columns'] = ohe_columns
    with open(os.path.join(ARTIFACTS_DIR, f'{city_name}_encoders.json'), 'w') as f:
        json.dump(encoders, f, indent=4)

    # Definicja i trening modelu Keras
    input_shape = (X.shape[1],)
    input_layer = layers.Input(shape=input_shape, name='input')
    x = layers.Dense(128, activation='relu')(input_layer)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    
    output_district = layers.Dense(len(encoders['district']['classes']), activation='softmax', name='district_output')(x)
    output_subdistrict = layers.Dense(len(encoders['subdistrict']['classes']), activation='softmax', name='subdistrict_output')(x)
    output_street = layers.Dense(len(encoders['street']['classes']), activation='softmax', name='street_output')(x)
    
    model = keras.Model(inputs=input_layer, outputs=[output_district, output_subdistrict, output_street])
    model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics={
        'district_output': 'accuracy',
        'subdistrict_output': 'accuracy',
        'street_output': 'accuracy'
    }
)
    
    print(f"Rozpoczynam trening na {len(train_data)} próbkach...")
    model.fit(X, [targets['district'], targets['subdistrict'], targets['street']], epochs=30, batch_size=32, verbose=0)
    
    model_path = os.path.join(ARTIFACTS_DIR, f'{city_name}_location_model.keras')
    model.save(model_path)
    print(f"Model dla {city_name} został zapisany w: {model_path}")


--- Krok 3: Trening Modeli ---

--- Trenowanie modelu dla miasta: Białystok ---
Pominięto Białystok z powodu zbyt małej ilości danych treningowych (0 ogłoszeń).

--- Trenowanie modelu dla miasta: Piaski ---
Pominięto Piaski z powodu zbyt małej ilości danych treningowych (0 ogłoszeń).

--- Trenowanie modelu dla miasta: Lipowa ---
Pominięto Lipowa z powodu zbyt małej ilości danych treningowych (0 ogłoszeń).

--- Trenowanie modelu dla miasta: Bydgoszcz ---
Pominięto Bydgoszcz z powodu zbyt małej ilości danych treningowych (0 ogłoszeń).

--- Trenowanie modelu dla miasta: Osówiec ---
Pominięto Osówiec z powodu zbyt małej ilości danych treningowych (0 ogłoszeń).

--- Trenowanie modelu dla miasta: Gdynia ---
Pominięto Gdynia z powodu zbyt małej ilości danych treningowych (0 ogłoszeń).

--- Trenowanie modelu dla miasta: Gdańsk ---
Pominięto Gdańsk z powodu zbyt małej ilości danych treningowych (0 ogłoszeń).

--- Trenowanie modelu dla miasta: Częstochowa ---
Pominięto Częstochowa z powodu zbyt

In [ ]:
# ==============================================================================
# KOMÓRKA 3: PREDYKCJA I GENEROWANIE WYNIKÓW
# ==============================================================================
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

print("\n--- Krok 4: Generowanie Predykcji i Wyników ---")

df_flats['Predict_Loc'] = df_flats['locationPath'].astype(str) # Domyślnie używamy oryginalnej ścieżki
predictions_made_count = 0

numerical_features = ['Area', 'Price', 'NumberOfRooms', 'Floor', 'Floors', 'BuiltYear']
categorical_features = ['BuldingType']
text_based_features = ['text_district_id', 'text_subdistrict_id', 'text_street_id']
similarity_based_features = ['similar_district_id', 'similar_subdistrict_id', 'similar_street_id']

for city_name in df_flats['city_name'].unique():
    if pd.isna(city_name): continue
    
    model_path = os.path.join(ARTIFACTS_DIR, f'{city_name}_location_model.keras')
    if not os.path.exists(model_path):
        print(f"Brak modelu dla {city_name}, pomijam predykcję.")
        continue
        
    print(f"Przetwarzam predykcje dla: {city_name}")
    
    # Wczytanie artefaktów
    model = keras.models.load_model(model_path)
    scaler = joblib.load(os.path.join(ARTIFACTS_DIR, f'{city_name}_scaler.gz'))
    with open(os.path.join(ARTIFACTS_DIR, f'{city_name}_encoders.json'), 'r') as f:
        encoders = json.load(f)

    df_to_predict = df_flats[(df_flats['city_name'] == city_name) & (df_flats['loc_5'] == 0)].copy()
    if df_to_predict.empty: continue

    # Przygotowanie cech wejściowych do predykcji
    X_pred_num = scaler.transform(df_to_predict[numerical_features])
    X_pred_cat_df = pd.get_dummies(df_to_predict[categorical_features], prefix='type')
    X_pred_cat_reindexed = X_pred_cat_df.reindex(columns=encoders['ohe_columns'], fill_value=0)
    
    X_pred = np.hstack([
        X_pred_num, 
        X_pred_cat_reindexed.values, 
        df_to_predict[text_based_features].values, 
        df_to_predict[similarity_based_features].values
    ])
    
    predictions = model.predict(X_pred, verbose=0)
    
    pred_district_idx = np.argmax(predictions[0], axis=1)
    pred_subdistrict_idx = np.argmax(predictions[1], axis=1)
    pred_street_idx = np.argmax(predictions[2], axis=1)

    df_to_predict['pred_loc_5'] = [encoders['district']['mapping'].get(str(i), 0) for i in pred_district_idx]
    df_to_predict['pred_loc_6'] = [encoders['subdistrict']['mapping'].get(str(i), 0) for i in pred_subdistrict_idx]
    df_to_predict['pred_loc_7'] = [encoders['street']['mapping'].get(str(i), 0) for i in pred_street_idx]
    
    predict_loc_series = df_to_predict.apply(lambda row: f"{int(row['loc_1'])},{int(row['loc_2'])},{int(row['loc_3'])},{int(row['loc_4'])},{int(row['pred_loc_5'])},{int(row['pred_loc_6'])},{int(row['pred_loc_7'])}", axis=1)
    df_flats.loc[df_to_predict.index, 'Predict_Loc'] = predict_loc_series
    predictions_made_count += len(df_to_predict)

print(f"\nWykonano predykcje dla {predictions_made_count} ogłoszeń.")

# --- Zapis i wyświetlenie wyników ---
output_filename = 'Location_Polska.csv'
df_flats.to_csv(output_filename, index=False)
print(f"Ostateczny plik został zapisany jako: {output_filename}")

print("\n--- 20 losowych przykładów predykcji ---")
predicted_rows = df_flats[df_flats['loc_5'] == 0].copy()

if not predicted_rows.empty:
    predicted_rows['pred_loc_5_name'] = predicted_rows['Predict_Loc'].apply(lambda x: lok_dict.get(int(x.split(',')[4]), 'Brak'))
    predicted_rows['pred_loc_7_name'] = predicted_rows['Predict_Loc'].apply(lambda x: lok_dict.get(int(x.split(',')[6]), 'Brak'))

    display_cols = ['city_name', 'Price', 'Area', 'locationPath', 'Predict_Loc', 'pred_loc_5_name', 'pred_loc_7_name', 'Description']
    
    with pd.option_context('display.max_colwidth', 100):
        display(predicted_rows.sample(min(20, len(predicted_rows)))[display_cols])
else:
    print("Nie znaleziono wierszy, dla których wykonano predykcję.")

print("\nZadanie zakończone pomyślnie!")


--- Krok 4: Generowanie Predykcji i Wyników ---
Brak modelu dla Białystok, pomijam predykcję.
Brak modelu dla Piaski, pomijam predykcję.
Brak modelu dla Lipowa, pomijam predykcję.
Brak modelu dla Bydgoszcz, pomijam predykcję.
Brak modelu dla Osówiec, pomijam predykcję.
Brak modelu dla Gdynia, pomijam predykcję.
Brak modelu dla Gdańsk, pomijam predykcję.
Brak modelu dla Częstochowa, pomijam predykcję.
Brak modelu dla Jasień, pomijam predykcję.
Przetwarzam predykcje dla: Łódź


C:\Users\Ai\AppData\Local\miniconda3\envs\projekt_stan\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Przetwarzam predykcje dla: Kraków


C:\Users\Ai\AppData\Local\miniconda3\envs\projekt_stan\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Brak modelu dla Lublin, pomijam predykcję.
Przetwarzam predykcje dla: Warszawa


...

Brak modelu dla Katowice, pomijam predykcję.
Brak modelu dla Radom, pomijam predykcję.
Brak modelu dla Sosnowiec, pomijam predykcję.
Brak modelu dla Siemianowice śląskie, pomijam predykcję.
Brak modelu dla Gliwice, pomijam predykcję.
Brak modelu dla Szczecin, pomijam predykcję.
Przetwarzam predykcje dla: Poznań


C:\Users\Ai\AppData\Local\miniconda3\envs\projekt_stan\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Brak modelu dla Toruń, pomijam predykcję.
Brak modelu dla Plewiska, pomijam predykcję.
Brak modelu dla Biedrusko, pomijam predykcję.
Brak modelu dla Bielawy, pomijam predykcję.
Brak modelu dla Olsztyn, pomijam predykcję.
Brak modelu dla Robakowo, pomijam predykcję.
Brak modelu dla Sopot, pomijam predykcję.
Brak modelu dla Opole, pomijam predykcję.
Brak modelu dla Mieroszów, pomijam predykcję.
Brak modelu dla Siedlce, pomijam predykcję.
Brak modelu dla Kętrzyn, pomijam predykcję.
Brak modelu dla Kłodzko, pomijam predykcję.
Brak modelu dla Dziwnówek, pomijam predykcję.
Brak modelu dla Dziwnów, pomijam predykcję.
Brak modelu dla Pobierowo, pomijam predykcję.
Brak modelu dla Iława, pomijam predykcję.
Brak modelu dla Szczecinek, pomijam predykcję.
Brak modelu dla Pabianice, pomijam predykcję.
Brak modelu dla Gniezno, pomijam predykcję.
Brak modelu dla Otwock, pomijam predykcję.
Brak modelu dla Kędzierzyn-koźle, pomijam predykcję.
Brak modelu dla Bielsko-biała, pomijam predykcję.
Brak modelu

C:\Users\Ai\AppData\Local\miniconda3\envs\projekt_stan\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


Brak modelu dla Szamotuły, pomijam predykcję.
Brak modelu dla Radzymin, pomijam predykcję.
Brak modelu dla Sobótka, pomijam predykcję.
Brak modelu dla Zakręt, pomijam predykcję.
Brak modelu dla Legnica, pomijam predykcję.
Brak modelu dla Sandomierz, pomijam predykcję.
Brak modelu dla Psarskie, pomijam predykcję.
Brak modelu dla Rokitki, pomijam predykcję.
Brak modelu dla Trzebiatów, pomijam predykcję.
Brak modelu dla Wrzosowa, pomijam predykcję.
Brak modelu dla Darłowo, pomijam predykcję.
Brak modelu dla Jelenia góra, pomijam predykcję.
Brak modelu dla Grójec, pomijam predykcję.
Brak modelu dla Piaseczno, pomijam predykcję.
Brak modelu dla Inowrocław, pomijam predykcję.
Brak modelu dla Dobczyce, pomijam predykcję.
Brak modelu dla Zielona góra, pomijam predykcję.
Brak modelu dla Racibórz, pomijam predykcję.
Brak modelu dla Polanica-zdrój, pomijam predykcję.
Brak modelu dla Ustka, pomijam predykcję.
Brak modelu dla Zalasewo, pomijam predykcję.
Brak modelu dla Malbork, pomijam predykcję.
